In [ ]:
import os
import pandas as pd
from langchain.llms import OpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
api_key = os.getenv("OPENAI_API_KEY")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

In [ ]:
pd.set_option('display.max_row',300)

In [ ]:
llm = OpenAI(
    temperature=0,
    model = "gpt-3.5-turbo-instruct"
)

tools = load_tools(["serpapi", "llm-math"], llm = llm)
agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)


In [ ]:
agent.run("你好，你是谁？")

In [ ]:
agent.run("现在的美国总统是谁？他的年龄乘以2等于多少？")

In [ ]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.chains import LLMMathChain
from langchain.prompts import  MessagesPlaceholder


In [ ]:
search = SerpAPIWrapper()
llm = ChatOpenAI(
    temperature = 0,
    model = "gpt-4o-mini"
)

llm_math_chain = LLMMathChain(
    llm = llm,
    verbose = True,
)

tools = [
    Tool(
        name = "Search",
        func = search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
    Tool(
        name = "Calculator",
        func = llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]
print(tools)

In [ ]:
memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages=True
)

agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    handle_parsing_errors=True,
    memory=memory 
    )

In [ ]:
print(agent_chain.agent.prompt.messages)
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])

In [ ]:
agent_chain.run("我叫CZL？")

In [ ]:
agent_chain.run("帮我计算一下372*234等于多少？")

In [ ]:
agent_chain.run("你还记得我叫什么名字吗")

In [ ]:
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    handle_parsing_errors=True,
    agent_kwargs={
        "extra_prompt_messages":[MessagesPlaceholder(variable_name="chat_history"),MessagesPlaceholder(variable_name="agent_scratchpad")],
    },
    memory=memory
    )

In [ ]:
agent_chain.run("好厉害，刚才我们都聊了什么？")